<a href="https://colab.research.google.com/github/doo52oh/Kaggle/blob/master/RNN_and_NLP_Detect_a_disaster_in_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ohdddd","key":"6d119e826d864d02a2b6057e0ef48b09"}'}

In [8]:
ls -1ha kaggle.json


kaggle.json


In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                               deadline             category            reward  teamCount  userHasEntered  
------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                                  2030-01-01 00:00:00  Getting Started  Knowledge       2801           False  
titanic                                           2030-01-01 00:00:00  Getting Started  Knowledge      20466            True  
house-prices-advanced-regression-techniques       2030-01-01 00:00:00  Getting Started  Knowledge       4886           False  
connectx                                          2030-01-01 00:00:00  Getting Started  Knowledge        374           False  
nlp-getting-started                               2030-01-01 00:00:00  Getting Started      Kudos       2351           False  
competitive-data-science-predict-future-sales     2020-12-31 23:59:00  Playground           Kudos       6610   

In [10]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 8.62MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 56.7MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 64.1MB/s]


In [11]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv


In [69]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [70]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth',-1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


Load Data



In [71]:
train = pd.read_csv('train.csv',
                         usecols=['text','target'],
                         dtype={'text':str, 'target':np.int64})

len(train)

7613

In [72]:
train['text'].head().values

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       '13,000 people receive #wildfires evacuation orders in California ',
       'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school '],
      dtype=object)

In [0]:
test = pd.read_csv('test.csv',
                   usecols=['text','id'],
                   dtype={'text':str, 'id':str})

** Mislabelled examples**

There are a number of examples in the training dataset that are mislabelled. The keyword can b used to find these

Thanks to Dmitri Kalyaevs whose notebook is where i found to do this : 
https://www.kaggle.com/dmitri9149/transformer-svm-semantically-identical-tweets



1.   항목 추가
2.   항목 추가



In [74]:
indices = [4415,4400,4399,4403,4397,4396,4394,4414,4393,4392,4404,4407,4420,4412,4408,4391,4405]
train.loc[indices]

,text,target
4415,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/xV3D9bPjHi #prebreak #best,1
4400,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/aAtt5aMnmD #prebreak #best,0
4399,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/nQiObcZKrT #prebreak #best,0
4403,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/qj3PVgaVN7 #prebreak #best,1
4397,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/J5onxFwLAo #prebreak #best,0
4396,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/wvTPuRYx63 #prebreak #best,0
4394,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/cx6auPneMu #prebreak #best,0
4414,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/cOMuiOk3mP #prebreak #best,1
4393,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/s4PNIhJQX7 #prebreak #best,0
4392,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/J2aQs5loxu #prebreak #best,1


In [75]:
train.loc[indices,'target']=0
train.loc[indices]

,text,target
4415,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/xV3D9bPjHi #prebreak #best,0
4400,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/aAtt5aMnmD #prebreak #best,0
4399,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/nQiObcZKrT #prebreak #best,0
4403,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/qj3PVgaVN7 #prebreak #best,0
4397,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/J5onxFwLAo #prebreak #best,0
4396,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/wvTPuRYx63 #prebreak #best,0
4394,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/cx6auPneMu #prebreak #best,0
4414,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/cOMuiOk3mP #prebreak #best,0
4393,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/s4PNIhJQX7 #prebreak #best,0
4392,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/J2aQs5loxu #prebreak #best,0


In [76]:
indices=[601,576,584,608,606,603,592,604,591,587]
train.loc[indices]

,text,target
601,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/c0p3SEsqWm via @usatoday,1
576,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/HQsU8LWltH via @usatoday,1
584,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday,0
608,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/4M5UHeyfDo via @USATODAY,1
606,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/hrqCJdovJZ,0
603,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/MqbYrAvK6h,1
592,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/pWAMG8oZj4,1
604,#FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/S4SiCMYRmH,1
591,#world FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/5zDbTktwW7,0
587,#world FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/wvExJjRG6E,1


In [0]:
train.loc[indices,'target']=1


In [78]:
indices = [3913,3914,3936,3921,3941,3937,3938,3136,3133,3930,3933,3924,3917]
train.loc[indices]

,text,target
3913,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/xxkHjySn0p http://t.co/JEVHKNJGBX,1
3914,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/jCDd6SD6Qn http://t.co/9gUCkjghms,0
3936,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/5xmCE6JufS http://t.co/3Zo7PX3p1V,0
3921,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/mTmoIa0Oo0 http://t.co/Nn4ZtCmSRU,0
3941,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/6zCfHi7Srw http://t.co/vWYkDaU1vm,0
3937,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/O097vSOtxk http://t.co/I23Xy7iEjj,0
3938,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/fDSaoOiskJ http://t.co/2uVmq4vAfQ,0
3136,Survival Kit Whistle Fire Starter Wire Saw Cree Torch Emergency Blanket S knife - Full reÛ_ http://t.co/2OroYUNYM2 http://t.co/C9JnXz3DXC,0
3133,Survival Kit Whistle Fire Starter Wire Saw Cree Torch Emergency Blanket S knife - Full reÛ_ http://t.co/cm7HqwWUlZ http://t.co/KdwAzHQTov,1
3930,2pcs 18W CREE Led Work Light Offroad Lamp Car Truck Boat Mining 4WD FLOOD BEAM - Full reaÛ_ http://t.co/O1SMUh2unn http://t.co/xqj6WgiuQH,0


In [0]:
train.loc[indices,'target']=0

In [80]:
indices=[246,270,266,259,253,251,250,271]
train.loc[indices]


,text,target
246,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... http://t.co/6LoJOoROuk via @Change,0
270,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/0fekgyBY5F via @Change,0
266,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/x2Wn7O2a3w via @Change,0
259,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/MatIJwkzbh via @Change,0
253,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... http://t.co/KPQk0C4G0M via @Change,1
251,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/sW1sBua3mN via @Change,1
250,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/m8MvDSPJp7 via @Change,0
271,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... http://t.co/SB5R7ShcCJ via @Change,1


In [0]:
train.loc[indices, 'target'] = 0

In [82]:
indices = [6119,6122,6123,6131,6160,6166,6167,6172,6212,6221,6230,6091,6108]
train.loc[indices]

,text,target
6119,That horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time,1
6122,The #Tribe just keeps sinking everyday it seems faster. As for this year it's been a titanic disaster.,1
6123,that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time,1
6131,'Amateur Night' Actress Reprises Role for 'Siren' - HorrorMovies.ca #horror http://t.co/W9Cd6OFfcj,1
6160,@SoonerMagic_ I mean I'm a fan but I don't need a girl sounding off like a damn siren,1
6166,Reasons I should have gone to Warped today: tony played issues showed up sleeping w sirens played attila is there issues issues issues,1
6167,Thu Aug 06 2015 01:20:32 GMT+0000 (UTC)\n#millcityio #20150613\ntheramin sirens,1
6172,my dad said I look thinner than usual but really im over here like http://t.co/bnwyGx6luh,1
6212,@PianoHands You don't know because you don't smoke. The way to make taxis and buses come is to light a cigarette to smoke while you wait.,1
6221,I get to smoke my shit in peace,1


In [0]:
train.loc[indices,'target']=0

In [84]:
indices = [7435,7460,7464,7466,7469,7475,7489,7495,7500,7525,7552,7572,7591,7599]
train.loc[indices]

,text,target
7435,Gunshot wound #9 is in the bicep. The only one of the ten wounds that is not in the chest/torso area. #KerrickTrial #JonathanFerrell,1
7460,@IcyMagistrate ÛÓher upper armÛÒ those /friggin/ icicle projectilesÛÒ and leg from various other wounds the girl looks like a miniature moreÛÓ,1
7464,Crawling in my skin\nThese wounds they will not hea,1
7466,Sorrower - Fresh Wounds Over Old Scars (2015 Death Metal) http://t.co/L056yj2IOi http://t.co/uTMWMjiRty,1
7469,@Captainn_Morgan car wreck ??,1
7475,Watertown Gazette owner charged in wreck http://t.co/JHc2RT0V9F,1
7489,@GeorgeFoster72 and The Wreck of the Edmund Fitzgerald,1
7495,Greer man dies in wreck http://t.co/n2qZbMZuly,1
7500,Omg if Cain dies i will be an emotional wreck #emmerdale,1
7525,The first piece of wreckage from the first-ever lost Boeing 777 which vanished back in early March along with the 239 people on board has,1


In [0]:
train.loc[indices,'target']=0

**Split training dataset**

To see if the model overfits the data during training I will take a slice of the training data as a validation dataset

In [0]:
val_data = train.tail(1500)
train_data = train.head(6113)

***Clean Text***

As with all datasets, text based data needs a bit of cleaning to. Some common cleaning steps are :



*   **Removing Noise :** so remove things that hold little meaning like URLs and html tags in the text. Punctuation is also usually removed at this stage though the Tensorflow tokenizer I use later does this by default so I leave out the logic at this step.
*   **Removing Stopwords :** Certain words like "a","the", and "are" are very common and hold little meaning for a sentence. Removing them speeds up training and helps with accuracy.
*  **Stemming or Lemmatization :** Many words are derived from a root or stem word. For example words like "working" and "worked" stem from the word "work". Reverting all words to their stem can help in some tasks though for this task I found that it made very little difference.

I've defined a few functions to perform this cleaning.

In [0]:
def remove_url(sentence):
  url = re.compile(r'https?://\S+\www\.\S+')
  return url.sub(r'', sentence)

def remove_at(sentence):
  url = re.compile(r'@\S+')
  return url.sub(r'', sentence)

def remove_html(sentence):
  html = re.compile(r'<.*?>')
  return html.sub(r'', sentence)

def remove_emoji(sentence):
  emoji = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji.sub(r'', sentence)

def remove_stopwords(sentence):
  words = sentence.split()
  words = [word for word in words if word not in stopwords.words('english')]

  return ' '.join(words)

In [0]:
stemmer = SnowballStemmer('english')

def stem_words(sentence):
  words = sentence.split()
  words = [stemmer.stem(word) for word in words]

  return ' '.join(words)



For speed I hav wrapped all of these cleaning functions into one. This is applied to all three datasets

In [0]:
def clean_text(data):
  data['text'] = data['text'].apply(lambda x : remove_url(x))
  data['text'] = data['text'].apply(lambda x : remove_at(x))
  data['text'] = data['text'].apply(lambda x : remove_html(x))
  data['text'] = data['text'].apply(lambda x : remove_emoji(x))
  data['text'] = data['text'].apply(lambda x : remove_stopwords(x))
  data['text'] = data['text'].apply(lambda x : stem_words(x))
  
  return data

In [90]:
train_data = clean_text(train_data)
val_data = clean_text(val_data)
test_data = clean_text(test)

train_data['text'].head().values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

array(['our deed reason #earthquak may allah forgiv us',
       'forest fire near la rong sask. canada',
       'all resid ask shelter place notifi officers. no evacu shelter place order expect',
       '13,000 peopl receiv #wildfir evacu order california',
       'just got sent photo rubi #alaska smoke #wildfir pour school'],
      dtype=object)

**Encode sentences**

A model will not understand what to do with a string representing a sentence. Instead it needs to be converted into an array of numbers representing the sentence. This is where a tokenizer is needed. A simple tokenizer will assign a number index to every unique word so that the model can treat it like a categorical value. Fundamentally this is what Tensorflows tokenizer does at this stage of preprocessing. 

To achieve this I'll define a couple of functions. The first defines the tokenizer by taking all the sentences from all three datasets and assinging an index number to every word present in the sentences. All three datasets are used here to ensure the tokenizer vocabulary includes all the wors present in the tweets. I have done a small piece of analysis in the appendix section at the bottom of this notebook to show how many words are in one of the datasets but not the others.

The second function uses the tokenizer to encode all the sentences into an array of index numbers representing the sentence. The second function also pads the sentences with zeros so that they are all the same size as the longest sentence in the training dataset. Note that the tokenizer reserves the index zero for this purpose.

In [0]:
def define_tokenizer(train_sentences, val_sentences, test_sentences):
  sentences = pd.concat([train_sentences, val_sentences, test_sentences])

  tokenizer = tf.keras.preprocessing.text.Tokenizer()
  tokenizer.fit_on_texts(sentences)

  return tokenizer

def encode(sentences, tokenizer):
  encoded_sentences = tokenizer.texts_to_sequences(sentences)
  encoded_sentences = tf.keras.preprocessing.sequence.pad_sequences(encoded_sentences, padding='post')

  return encoded_sentences
  

In [0]:
tokenizer = define_tokenizer(train_data['text'],val_data['text'], test_data['text'])

encoded_sentences = encode(train_data['text'], tokenizer)
val_encoded_sentences = encode(val_data['text'], tokenizer)
encoded_test_sentences = encode(test_data['text'], tokenizer)


The tokeniser provides some interesting information about the sentences  it encodes. To get the index number assinged to a word i can look up the word in the tokenizers word index(which is just a python dict with the words as keys and the indext numbers as values).

In [94]:
tokenizer.word_index['disaster']



743

The word index can also be used to find out how many words are in the vacab

In [95]:
len(tokenizer.word_index)

23114

As well as some configurations that are used to tokenize sentences such as whether the tokenizer changes all characters to lowercase, what split it performs to get the words from the sentences and what characters it filers out.

In [96]:
print('Lower :', tokenizer.get_config()['lower'])
print('Split :', tokenizer.get_config()['split'])
print('Filters :', tokenizer.get_config()['filters'])

Lower : True
Split :  
Filters : !"#$%&()*+,-./:;<=>?@[\]^_`{|}~	



**Import GloVe Embedding**

While I could train my own owrd embedding for the model it might help to use a pre-trained word embedding. This enables me to take advantage of an embedding that has unone more rogourous training. Additionally it will also include words that I may not have in my training dataset (but may appear in the test dataset) which helps with overfitting.

The first thing to do then is to load the embedding. I'll use GloVe for this task.

In [0]:
embedding_dict = {}

with open('glove.6B.100d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:],'float32')
        embedding_dict[word] = vectors
        
f.close()


To ensure the encoding of the tokenizer and the embeddings are synchronized I use the below function to update the encoded words in the embedding with the encoding from the tokenizer.

In [112]:
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, 100))

for word, i in tokenizer.word_index.items():
    if i > num_words:
        continue
    
    emb_vec = embedding_dict.get(word)
    
    if emb_vec is not None:
        embedding_matrix[i] = emb_vec

ValueError: ignored

**Define pipeline**

With the sentences encoded they can now be prepared to be fed into the model. Tensorflow provides an api to format data in its own format. While data can be inserted in a more common format(such as numpy arrays), tensorflow seems to prefer its own format and provides a few handy bits of functionality as incentives.

Firstly then I will convert the encoded sentences and lbels into tensors.

In [0]:
tf_data = tf.data.Dataset.from_tensor_slices((encoded_sentences, train_data['target'].values))

Now the data is in the tensorflow format a few handy methods can be added to imporve the training. This includes shuffling the data per training step, processing the next batch of data for training while the current batch of data is training and defining each batch as a padded batch.

In [0]:
def pipeline(tf_data, buffer_size=100, batch_size=32):
  tf_data = tf_data.shuffle(buffer_size)
  tf_data = tf_data.prefetch(tf.data.experimental.AUTOTUNE)
  df_Data = tf_data.padded_batch(batch_size, padded_shapes=([None],[]))

  return tf_data

tf_data = pipeline(tf_data, buffer_size=1000, batch_size=32)


In [0]:
print(tf_data)

A similar pipleline is defined for the validation dataset. The difference is the lack of shuffling to speed up the validatiaon.

In [0]:
tf_val_data = tf.data.Dataset.from_tensor_slices((val_encoded_sentences, val_data['target'].values))


In [0]:
def val_pipeline(tf_data, batch_size=1):
  tf_data = tf_data.prefetch(tf.data.experimental.AUTOTUNE)
  tf_data = tf_data.padded_batch(batch_size, padded_shapes = ([None],[]))

  return tf_data

tf_val_data = val_pipeline(tf_val_data, batch_size = len(val_data))


In [0]:
print(tf_val_data)

**Train Model**
Now I am ready to define and rain the model. Firstly I will define the model:

*  **Embedding layer :** To enable the model to gain an understanding of a words meaning it will need a word embedding. This embedding creates a number of generic features against each word that could represent anything from a words gender to whether it is has a positive or negative sentiment. This also enables the model to identify relationships between words. This page from Tensorflows documnetation contains a good diagram of what one might look like

*  **RNN layer :** An RNN layer is a bit complex to explain in it's entirely in this notebook. Check out the course link at the top of this notebook for a full run through of the theory. Tensorflow offers all three of the major RNN layers; simple RNN, GRU and LSTM. I tried all of these layers and found that the LSTM layer worked best here.

*  **Dense Layer :** This final layer takes the LSTM output and applies a class to the sentence i.e. a 1 if the sentence is reporting a real disaster or 0 if not

In [0]:
embedding = tf.keras.layers.Embedding(
    len(tokenizer.word_index) +1,
    100,
    embeddings_initailizer = tf.keras.initializers.Constant(embedding_matrix),
    trainable = True
)

In [0]:
model = tf.keras.Sequential([embedding, 
                             tf.keras.layers.SpatialDropout1D(0,2),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout0.2, recurrent_dropout=0.2)),
                             tf.keras.layers.Dense(1,activation='sigmoid')
                             ])

Then compile the model defining the training function (adam) and the loss function (log loss). I have also added a metrics parameter so that the models accuracy is printed per epoch.

In [0]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=['accuracy','Precision','Recall']
)

To avoid the model stepping over the optimum I'll add learning rate decay logic to reduce the learning rate if the loss plateaus for two or more epochs. I'll also add early stopping if loss hasn't fallen for five epochs. This saves some processing.

In [0]:
callbacks =[
            tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, verbose=1),
            tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1),
]

training model

In [0]:
history = model.fit(
    tf_data,
    validation_data = tf_val_data,
    epochs = 50,
    callbacks = callbacks
)

**Evaluate**

Let's take a look at the models output to get an idea how it did. The quickest and easiest evaluation method is to take a look at the metrics produced by the model. The final metrics can be extracted using the evaluate method. Since this competition uses an F1 score to rank submissions it may be worth having a look at it on the training dataset.

In [0]:
metrics = model.evaluate(tf_val_data)

precision = metrics[2]
recall = metrics[3]
f1 = 2*(precision*recall)/(precision+recall)

print('F1 score : ' + str(f1))

Additionally the metrics produced per epoch when the model was training can be visualised to get a better idea for how the trianing went.

In [0]:
fig, axs = plt.subpltos(1,4, figsize(20,5))

axs[0].set_title('Loss')
axs[0].plot(history.history['loss'], label='train')
axs[0].plot(history.history['val_loss'], label='val')
axs[0].legend()

axs[1].set_title('Accuracy')
axs[1].plot(history.history['accuracy'], label='train')
axs[1].plot(history.history['val_accuracy'], label='val')
axs[1].legend()

axs[2].set_title('Precision')
axs[2].plot(history.history['Precision'], label='train')
axs[2].plot(history.history['val_Precision'], label='val')
axs[2].legend()

axs[3].set_title('Recall')
axs[3].plot(history.history['Recall'], label='train')
axs[3].plot(history.history['val_Reall'],label='val')
axs[3].legend()

It is also worth having a look at which sentences the model got wrong. To do this the model needs to produce predictions for the training dataset. This involves a slightly different pipeline.

In [0]:
predictions = model.predict(tf_val_data)
predictions = np.concatenate(predictions).round().astype(int)

val_data['predictions'] = predictions

First take a look at the flase positives (when the model thought there was a disaster in the tweet but there was not).

In [0]:
false_positives = val_data[(val_data['predictions']==1)&(val_data['target']==0)]

print('Count of flase positives: ' + str(len(false_positives)))


In [0]:
false_positives.head(10)

And then do the smae with the flase negatives( when the model didnt think there was a disaster in a tweet when in fact there was)

In [0]:
false_negatives = val_data[(val_Data['predictions']==0)&(val_data['target']==1)]

print('count of false negatives: ' + str)len(false_negatives)

In [0]:
false_positives.tail(10)

**Submission**

With the model trained it now takes a few more steps to load the test data and use the model to label the test sentences as either diaster or no disaster. First convert the data to tensorflow dataset and apply the pipeline methods. The pipleline has been adjusted slightly to account for not wanting any shuffling and the different shape of the inpu (no label).

In [0]:
tf_test_data = tf.data.Dataset.from_tensor_slices((encoded_test_sentences))


In [0]:
def test_pipeline(tf_data, batch_size=1):
  tf_data = tf_data.prefetch(tf.data.experimental.AUTOUNE)
  tf_data = tf_Data.padded_batch(batch_size, padded_shapes=([None]))

  return tf_data

tf_test_data = test_pipeline(tf_test_data)

Then use the model to apply labels to the test data.

In [0]:
predictions = model.predict(tf_test_data)

The model outputs a probability per sentence. The easy way to set a threshold of 0.5(i.e. if the probability is less than 0.5 set the label to 0 and vice versa) is to use the round method.

In [0]:
predictions = np.concatentate(predictions).round().astype(int)


Write the submission to a csv file.

In [0]:
submission = pd.DataFrame(data={'target'}: predictions, index=test_data['id'])
submission.index = submission.index.rename('id')
submission.to_csv('submission.csv')

In [0]:
submission.head()